# Time correlation for real artworks from different models.

In [1]:
from scipy.io import loadmat
import re
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd
from scipy.stats import pearsonr
import pickle
import os
from pathlib import Path

pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
pruned_mat_file='/home/khanff/waga/WAGA/analyses_notebooks/datasets/processed/groundtruth_pruned.mat'
mat_file = loadmat(pruned_mat_file)

In [3]:
years = []
re_string = '[0-9][0-9][0-9][0-9]\.'
for m in mat_file['groundtruth_pruned'][0][0][0]:
    text = re.findall(re_string, m[0][0])
    if text != []:
        year = int(text[0][:-1])
        if year < 1400 or year > 2010:
            years.append(np.nan)
        else:
            years.append(int(year))
    else:
        years.append(np.nan)
years = np.array(years)

In [4]:
indexes = ~np.isnan(years)

In [5]:
df_1k = pd.read_csv('./datasets/processed/combined_wofflins_real.csv')
df_5k = pd.read_csv('./datasets/processed/real_wofflin_scores_combined_normalised.csv')
combined = [df_1k, df_5k]
df = pd.concat(combined)
df['name'] = df['Input.image'].apply(lambda x: Path(x).name)
files_im = [Path(f[0][0]).name for f in mat_file['groundtruth_pruned'][0][0][0]]
real_indexes = [files_im.index(f) for f in df.name.values]

In [6]:
def get_principal_components(all_feats, n_components, verbose=False):
    pca = PCA(n_components=n_components, svd_solver='full')
    feats = pca.fit_transform(all_feats)
    if verbose:
        print(pca.explained_variance_ratio_)
        print(np.cumsum(pca.explained_variance_ratio_))
    return feats, len(pca.explained_variance_ratio_)

def get_coefficients(feats, years, n_components, coefficient, file, verbose=False):
    pc, components = get_principal_components(feats, n_components, verbose=verbose)
    components_to_return = components
    components = min(components, 30)
    if verbose:
        print(f"{components} components required")
    
    for c in range(components):
        coefficients.setdefault(file,[]).append(pearsonr(years, pc[:, c])[0])
#         p_values.setdefault(file,[]).append(pearsonr(years, pc[:, c])[1])
    return pd.DataFrame(coefficients), components_to_return

In [7]:
def get_r(a):
    rho = np.sqrt(a[0]**2 + a[-1]**2)
    return rho

def get_theta(a):
    phi = np.arctan2(a[-1], a[0])
    return phi

def get_coefficients_radial(feats, years, n_components,coefficients, file, verbose=False):
    feats, ratio = get_principal_components(feats,n_components, verbose=verbose)

    radial_feats_r = np.apply_along_axis(get_r, 1, np.asarray(feats[:,:2]))
    radial_feats_theta = np.apply_along_axis(get_theta, 1, np.asarray(feats[:,:2]))
    x = np.expand_dims(radial_feats_theta, axis=0)
    y = np.expand_dims(radial_feats_r, axis=0)
    ang_feats = np.concatenate((x, y), axis=0)
    coefficients = {}
    for c in range(n_components):
        coefficients.setdefault(file,[]).append(pearsonr(years, ang_feats.T[:, c])[0])        
    return pd.DataFrame(coefficients)

In [8]:
folder = './datasets/features/real_art_features'
# folder = '/home/khanff/eccv/ViT-pytorch/features/real/'
principles = ['Linearly-vs-Painterly', 'Planar-vs-Recessional',
       'closed-form-vs-open-form', 'multiplicity-vs-unity',
       'absolute-clarity-vs-relative-clarity']
table_for_n_components = {}
dataframes = {}
files = os.listdir(folder)
for file in files:
    coefficients = {}
    p_values = {}
    with open(os.path.join(folder, file), 'rb') as f:
        feats = pickle.load(f)
    coefficient_time = get_coefficients(feats[indexes], years[indexes], 0.95, coefficients, file, False)
    radial_coefficient_time = get_coefficients_radial(feats[indexes], years[indexes], 2, coefficients, file, False)
    abs_coefficient_time = [abs(i) for i in coefficient_time]
    print(file)
    print('PCC: ',abs_coefficient_time[0].max().values,' Radial PCC: ',abs(radial_coefficient_time).max().values)  

resnet101_plus2_real.pkl
PCC:  [0.06498374]  Radial PCC:  [0.05128971]
resnet101_pretrained_real.pkl
PCC:  [0.5393042]  Radial PCC:  [0.23968259]
resnet50_plus2_real.pkl
PCC:  [0.0824481]  Radial PCC:  [0.0521291]
resnet50_pretrained_real.pkl
PCC:  [0.5432301]  Radial PCC:  [0.24222095]
StyleCWAN1_real.pkl
PCC:  [0.38887915]  Radial PCC:  [0.1600193]
StyleCWAN2_real.pkl
PCC:  [0.31629225]  Radial PCC:  [0.11427436]
vgg16_plus2_real.pkl
PCC:  [0.33889238]  Radial PCC:  [0.13143862]
vgg16_pretrained_real.pkl
PCC:  [0.52496757]  Radial PCC:  [0.21482059]
StyleGAN2_real.pkl
PCC:  [0.23529744]  Radial PCC:  [0.10633537]
vit_real.pkl
PCC:  [0.63141546]  Radial PCC:  [0.30625501]
StyleCAN1_real.pkl
PCC:  [0.31692972]  Radial PCC:  [0.20750769]
StyleCAN2_real.pkl
PCC:  [0.30725605]  Radial PCC:  [0.24041198]
StyleGAN1_real.pkl
PCC:  [0.19523487]  Radial PCC:  [0.10557771]
